In [20]:

!pip3 install -r requirements.txt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.utils import shuffle
from keras.models import model_from_json
from keras.models import load_model
import pickle
import pandas as pd
import numpy as np
import csv

In [0]:

''' Firstly, I am reading my train_image and train_label with the help of pickle and pandas, and then I am converting each of them
into numpy array and then dividing my training data into training and validation set(7800 training data and 200 validation),and
then reshaping them into 4d matrics for inputing this data into our neural-network architecture, and I had shuffled the data
to get more randomness in it'''
train_x = pd.read_pickle("train_image.pkl")

train_y = pd.read_pickle("train_label.pkl")

X = np.array(train_x)
y = np.array(train_y)

X, y = shuffle(X, y)
x_train = X[:7800]
x_test   = X[7800:]
y_train = y[:7800]
y_test   = y[7800:]

x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [0]:
'''I used batch-size of 10 and 10 epochs for our training data, then converted each data into float type and divided it by
255 as it is the highest RGB value for a pixel, and then I converted our output from array of labeled data
(from 0 to nb_classes-1) to one-hot vector'''
input_shape = (28,28,1)
batch_size = 128
epochs = 50

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

num_classes = 7

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:

'''Its time to create our CNN architecture, the one which I was using contains 2 convolutional layer, 1 Maxpooling layer, 
2 dense layer, 1 flatten layer and 2 dropouts''' 
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [28]:
'''I have categorical_crossentropy as my loss function and adadelta optimizer, and we are fitting the training as well as the
validation data'''
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
__________

In [29]:
# The score is evaluated on the basis of performance on validation data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3818643593788147
Test accuracy: 0.92


In [0]:
# Here I am saving my model and my model weights
model_json = model.to_json()

with open("model_num.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_num.h5")

In [31]:
# Here I am reading the test data using pandas and pickle
test_x = pd.read_pickle("test_image.pkl")
X = np.array(test_x)

print(X.shape)
x_test = X.reshape(-1,28,28,1)

(2000, 784)


In [32]:
# Here I am loading my previous made CNN Model and its weights
json_file = open('model_num.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model_num.h5")
print("Loaded model from disk")

loaded_model.save('model_num.hdf5')
loaded_model=load_model('model_num.hdf5')

Loaded model from disk


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [33]:
''' At last I am using the test data to predict the classes of the images in it and save it in a CSV file named
himanchal_chandra.csv '''
a = np.arange(200)
b = loaded_model.predict_classes(x_test)

b[:200]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 6, 0, 6, 6, 0, 0, 0,
       6, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 6, 0, 3, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 2, 0, 6, 6, 0,
       0, 0, 0, 0, 3, 0, 6, 6, 0, 0, 0, 0, 0, 0, 3, 6, 6, 0, 0, 6, 0, 6,
       0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 6, 6, 0, 3, 0, 0, 0, 0, 0, 6, 0, 6, 6, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 6, 0, 0, 0, 0, 0, 6, 0, 6, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0])

In [0]:
data1 = ['image_index','class']
data2 = a.T
data3 = b.T  

list1 = []
for index, i in enumerate(data2):
	list2 = [i, data3[index]]
	list1.append(list2)

with open('himanchal_chandra.csv', 'w+', newline='') as f:
	wr = csv.writer(f)
	wr.writerow(data1)
	wr.writerows(list1)